In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from keras.models import load_model
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings('ignore')

2024-03-28 03:18:04.701187: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 03:18:04.701229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 03:18:04.701266: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-28 03:18:04.711227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 03:18:06.888270: W tensorflow/c

In [2]:
# Load data
test = pd.read_csv('./DATA/quenn.csv')
feature = pd.read_feather('./featurebank/feature_esm2.feather')
test = test.merge(feature,on='uniprot_id',how='left')

In [3]:
# Load model
model_location = "./model/QUEEN_MLPmodel_final.pkl"
with open(model_location, "rb") as f:
  QUEEN_model = pickle.load(f)
  
# Queen pred
y_test = QUEEN_model.predict(np.array(test.iloc[:,3:]))
inv_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 10, 9: 12, 10: 14, 11: 24}

y_test_transformed = np.array([inv_map[x] for x in y_test])
print("These are the predicted labels:")
print(y_test_transformed)

These are the predicted labels:
[1 1 2 ... 2 2 1]


In [4]:
res = test[['uniprot_id','label']]
res['pred'] = y_test_transformed

In [5]:
res = pd.DataFrame(classification_report(res["label"].values, res["pred"].values, labels=list(set(res.label)),zero_division=0, output_dict=True)).T
res.to_csv("output/queen_report.csv")
res

,precision,recall,f1-score,support
1,0.606737,0.773979,0.680230,4212.0
2,0.837628,0.710824,0.769034,10153.0
3,0.639175,0.710828,0.673100,785.0
4,0.732531,0.689760,0.710503,2295.0
5,0.914894,0.304965,0.457447,141.0
6,0.599159,0.699673,0.645527,1222.0
7,0.250000,0.333333,0.285714,3.0
8,0.150327,0.300654,0.200436,153.0
10,0.864078,0.868293,0.866180,205.0
12,0.361446,0.937500,0.521739,96.0
